In [12]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('Datasets/df_final_eda.csv')

df_ml = pd.DataFrame(df)

In [3]:
df_ml.head()

,Unnamed: 0,user_id_x,Item_id,Item_name,Hours_Played,genres,app_name,release_year,id,user_id_y,Recommend,Year_Posted,sentiment_analysis
0,0,76561198035860222,65930,The Bureau: XCOM Declassified,0.00,Action,The Bureau: XCOM Declassified,2013,65930,longinus,False,2014,0
1,1,76561198035860222,65930,The Bureau: XCOM Declassified,0.00,Action,The Bureau: XCOM Declassified,2013,65930,manuel_vega182002,True,2015,2
2,2,GGargula,65930,The Bureau: XCOM Declassified,13.57,Action,The Bureau: XCOM Declassified,2013,65930,longinus,False,2014,0
3,3,GGargula,65930,The Bureau: XCOM Declassified,13.57,Action,The Bureau: XCOM Declassified,2013,65930,manuel_vega182002,True,2015,2
4,4,76561198085547365,65930,The Bureau: XCOM Declassified,0.00,Action,The Bureau: XCOM Declassified,2013,65930,longinus,False,2014,0


In [4]:
df_final_ml = df_ml[['genres', 'app_name','Item_name','Hours_Played', 'release_year','sentiment_analysis']]

In [ ]:
df_final_ml.shape

In [ ]:
sample_df = df_final_ml.sample(2500).reset_index(drop=True)  ## Se tomo una muestra de 2500 datos para el sistema de recomendacion

In [ ]:
# Se convierte el DataFrame a archivo csv
sample_df.to_csv("Datasets/Machine_learning.csv")